# Quick Start

This notebook demonstrates how to use MARO's reinforcement learning (RL) toolkit to solve the container inventory management ([CIM](https://maro.readthedocs.io/en/latest/scenarios/container_inventory_management.html)) problem. It is formalized as a multi-agent reinforcement learning problem, where each port acts as a decision agent. The agents take actions independently, e.g., loading containers to vessels or discharging containers from vessels.   

## [State Shaper](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#shapers)

State shaper converts the environment observation to the model input state which includes temporal and spatial information. For this scenario, the model input state includes: 

- Temporal information, it includes the past week's information of ports and vessels, such as shortage on port and remaining space on vessel. 

- Spatial information, it includes the related downstream port features.    

In [1]:
import numpy as np
from maro.rl import StateShaper


class CIMStateShaper(StateShaper):
    def __init__(self, *, look_back, max_ports_downstream, port_attributes, vessel_attributes):
        super().__init__()
        self._look_back = look_back
        self._max_ports_downstream = max_ports_downstream
        self._port_attributes = port_attributes
        self._vessel_attributes = vessel_attributes
        self._dim = (look_back + 1) * (max_ports_downstream + 1) * len(port_attributes) + len(vessel_attributes)

    def __call__(self, decision_event, snapshot_list):
        tick, port_idx, vessel_idx = decision_event.tick, decision_event.port_idx, decision_event.vessel_idx
        ticks = [tick - rt for rt in range(self._look_back-1)]
        future_port_idx_list = snapshot_list["vessels"][tick: vessel_idx: 'future_stop_list'].astype('int')
        port_features = snapshot_list["ports"][ticks: [port_idx] + list(future_port_idx_list): self._port_attributes]
        vessel_features = snapshot_list["vessels"][tick: vessel_idx: self._vessel_attributes]
        state = np.concatenate((port_features, vessel_features))
        return str(port_idx), state
    
    @property
    def dim(self):
        return self._dim

## [Action Shaper](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#shapers)

Action shaper is used to convert an agent's model output to an environment executable action. For this specific scenario, the output is a discrete index that corresponds to a percentage indicating the fraction of containers to be loaded to or discharged from the arriving vessel.

In [2]:
from maro.rl import ActionShaper
from maro.simulator.scenarios.cim.common import Action


class CIMActionShaper(ActionShaper):
    def __init__(self, action_space):
        super().__init__()
        self._action_space = action_space
        self._zero_action_index = action_space.index(0)

    def __call__(self, model_action, decision_event, snapshot_list):
        assert 0 <= model_action < len(self._action_space)
        
        scope = decision_event.action_scope
        tick = decision_event.tick
        port_idx = decision_event.port_idx
        vessel_idx = decision_event.vessel_idx
        port_empty = snapshot_list["ports"][tick: port_idx: ["empty", "full", "on_shipper", "on_consignee"]][0]
        vessel_remaining_space = snapshot_list["vessels"][tick: vessel_idx: ["empty", "full", "remaining_space"]][2]
        early_discharge = snapshot_list["vessels"][tick:vessel_idx: "early_discharge"][0]
     
        if model_action < self._zero_action_index:
            # The number of loaded containers must be less thean the vessel's remaining space.
            actual_action = max(round(self._action_space[model_action] * port_empty), -vessel_remaining_space)
        elif model_action > self._zero_action_index:
            # In the case of an early discharge event, we need to subtract the early discharge amount from the expected 
            # discharge quote.   
            plan_action = self._action_space[model_action] * (scope.discharge + early_discharge) - early_discharge
            actual_action = round(plan_action) if plan_action > 0 else round(self._action_space[model_action] * scope.discharge)
        else:
            actual_action = 0

        return Action(vessel_idx, port_idx, actual_action)

## [Experience Shaper](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#shapers)

Experience shaper is used to convert an episode trajectory to trainable experiences for RL agents. For this specific scenario, the reward is a linear combination of fulfillment and shortage in a limited time window.

In [3]:
from collections import defaultdict

from maro.rl import ExperienceShaper


class TruncatedExperienceShaper(ExperienceShaper):
    def __init__(self, *, time_window: int, time_decay_factor: float, fulfillment_factor: float,
                 shortage_factor: float):
        super().__init__(reward_func=None)
        self._time_window = time_window
        self._time_decay_factor = time_decay_factor
        self._fulfillment_factor = fulfillment_factor
        self._shortage_factor = shortage_factor

    def __call__(self, trajectory, snapshot_list):
        experiences_by_agent = {}
        for i in range(len(trajectory) - 1):
            transition = trajectory[i]
            agent_id = transition["agent_id"]
            if agent_id not in experiences_by_agent:
                experiences_by_agent[agent_id] = defaultdict(list)
            
            experiences = experiences_by_agent[agent_id]
            experiences["state"].append(transition["state"])
            experiences["action"].append(transition["action"])
            experiences["reward"].append(self._compute_reward(transition["event"], snapshot_list))
            experiences["next_state"].append(trajectory[i+1]["state"])

        return experiences_by_agent

    def _compute_reward(self, decision_event, snapshot_list):
        start_tick = decision_event.tick + 1
        end_tick = decision_event.tick + self._time_window
        ticks = list(range(start_tick, end_tick))

        # Calculate truncate reward.
        future_fulfillment = snapshot_list["ports"][ticks::"fulfillment"]
        future_shortage = snapshot_list["ports"][ticks::"shortage"]
        decay_list = [self._time_decay_factor ** i for i in range(end_tick - start_tick)
                      for _ in range(future_fulfillment.shape[0]//(end_tick-start_tick))]

        tot_fulfillment = np.dot(future_fulfillment, decay_list)
        tot_shortage = np.dot(future_shortage, decay_list)

        return np.float(self._fulfillment_factor * tot_fulfillment - self._shortage_factor * tot_shortage)

## [Agent](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#agent)

For this scenario, the agent is the abstraction of a port. We choose DQN as our underlying learning algorithm with a TD-error-based sampling mechanism.  

In [4]:
from maro.rl import AbsAgent, ColumnBasedStore


class CIMAgent(AbsAgent):
    def __init__(self, name, algorithm, experience_pool: ColumnBasedStore, min_experiences_to_train,
                 num_batches, batch_size):
        super().__init__(name, algorithm, experience_pool)
        self._min_experiences_to_train = min_experiences_to_train
        self._num_batches = num_batches
        self._batch_size = batch_size

    def train(self):
        if len(self._experience_pool) < self._min_experiences_to_train:
            return

        for _ in range(self._num_batches):
            indexes, sample = self._experience_pool.sample_by_key("loss", self._batch_size)
            state = np.asarray(sample["state"])
            action = np.asarray(sample["action"])
            reward = np.asarray(sample["reward"])
            next_state = np.asarray(sample["next_state"])
            loss = self._algorithm.train(state, action, reward, next_state)
            self._experience_pool.update(indexes, {"loss": loss})

## [Agent Manager](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#agent-manager)

The agent manager inherits from MARO's `AbsAgentManager` which is an agent assembler and isolates the complexities of the environment and algorithm. It will load the DQN algorithm and an experience pool for each agent.

In [5]:
import io
import yaml

from torch.nn.functional import smooth_l1_loss
from torch.optim import RMSprop

from maro.rl import AbsAgentManager, LearningModel, MLPDecisionLayers, DQN, DQNHyperParams, ColumnBasedStore


num_actions = 21


class DQNAgentManager(AbsAgentManager):
    def _assemble(self, agent_dict):
        for agent_id in self._agent_id_list:
            eval_model = LearningModel(decision_layers=MLPDecisionLayers(name=f'{agent_id}.policy',
                                                                         input_dim=self._state_shaper.dim,
                                                                         output_dim=num_actions,
                                                                         hidden_dims=[256, 128, 64],
                                                                         dropout_p=.0)
                                       )

            algorithm = DQN(model_dict={"eval": eval_model},
                            optimizer_opt=(RMSprop, {"lr": 0.05}),
                            loss_func_dict={"eval": smooth_l1_loss},
                            hyper_params=DQNHyperParams(num_actions=num_actions, reward_decay=.0,
                                                        num_training_rounds_per_target_replacement=5, tau=0.1)
                           )

            experience_pool = ColumnBasedStore()
            
            agent_dict[agent_id] = CIMAgent(name=agent_id, algorithm=algorithm, experience_pool=experience_pool,
                                            min_experiences_to_train=1024, num_batches=10, batch_size=128)

    def store_experiences(self, experiences):
        # The output of the experience shaper is organized by the agent ID.   
        for agent_id, exp in experiences.items():
            exp.update({"loss": [1e8] * len(exp[next(iter(exp))])})
            self._agent_dict[agent_id].store_experiences(exp)

## Main Loop with [Actor and Learner](https://maro.readthedocs.io/en/latest/key_components/rl_toolkit.html#learner-and-actor)

This code cell demonstrates the typical workflow of a learning policy's interaction with a MARO environment. 

- Initialize an environment with specific scenario and topology parameters. 

- Define scenario-specific components, e.g. shapers. 

- Create an agent manager, which assembles underlying agents. 

- Create an actor and a learner to start the training process in which the agent manager interacts with the environment for collecting experiences and updating policies. 

In [6]:
from maro.simulator import Env
from maro.rl import SimpleLearner, SimpleActor, AgentMode, TwoPhaseLinearExplorer
from maro.utils import Logger, LogFormat

# Step 1: initialize a CIM environment for using a toy dataset. 
env = Env("cim", "toy.4p_ssdd_l0.0", durations=1120)
total_episodes = 100
agent_id_list = [str(agent_id) for agent_id in env.agent_idx_list]

# Step 2: create state, action and experience shapers. We also need to create an explorer here due to the 
# greedy nature of the DQN algorithm.  
state_shaper = CIMStateShaper(look_back=7, max_ports_downstream=2, 
                              port_attributes=["empty", "full", "on_shipper", "on_consignee", 
                                               "booking", "shortage", "fulfillment"],
                              vessel_attributes=["empty", "full", "remaining_space"]
                             )

action_shaper = CIMActionShaper(action_space=list(np.linspace(-1.0, 1.0, num_actions)))

experience_shaper = TruncatedExperienceShaper(time_window=100, fulfillment_factor=1.0, shortage_factor=1.0,
                                              time_decay_factor=0.97)

explorer = TwoPhaseLinearExplorer(agent_id_list, total_episodes, 
                                  epsilon_range_dict={"_all_": (.0, .4)},
                                  split_point_dict={"_all_": (.5, .8)},
                                  with_cache=True)

# Step 3: create an agent manager.
agent_manager = DQNAgentManager(name="cim_learner",
                                mode=AgentMode.TRAIN_INFERENCE,
                                agent_id_list=agent_id_list,
                                state_shaper=state_shaper,
                                action_shaper=action_shaper,
                                experience_shaper=experience_shaper,
                                explorer=explorer)

# Step 4: Create an actor and a learner to start the training process. 
actor = SimpleActor(env, agent_manager)
learner = SimpleLearner(trainable_agents=agent_manager, actor=actor,
                        logger=Logger("single_host_cim_learner", format_=LogFormat.simple, auto_timestamp=False))

learner.train(total_episodes)

10:42:49 | single_host_cim_learner | INFO | ep 1 - performance: {'order_requirements': 2240000, 'container_shortage': 1471903, 'operation_number': 2998694}, epsilons: {'0': 0.4, '1': 0.4, '2': 0.4, '3': 0.4}
10:42:53 | single_host_cim_learner | INFO | ep 2 - performance: {'order_requirements': 2240000, 'container_shortage': 1749783, 'operation_number': 2682255}, epsilons: {'0': 0.39836734693877884, '1': 0.39836734693877884, '2': 0.39836734693877884, '3': 0.39836734693877884}
10:42:57 | single_host_cim_learner | INFO | ep 3 - performance: {'order_requirements': 2240000, 'container_shortage': 1602611, 'operation_number': 2882583}, epsilons: {'0': 0.39673469387755766, '1': 0.39673469387755766, '2': 0.39673469387755766, '3': 0.39673469387755766}
10:43:01 | single_host_cim_learner | INFO | ep 4 - performance: {'order_requirements': 2240000, 'container_shortage': 1685691, 'operation_number': 2560449}, epsilons: {'0': 0.3951020408163365, '1': 0.3951020408163365, '2': 0.3951020408163365, '3': 

10:45:12 | single_host_cim_learner | INFO | ep 32 - performance: {'order_requirements': 2240000, 'container_shortage': 538322, 'operation_number': 3954792}, epsilons: {'0': 0.3493877551021435, '1': 0.3493877551021435, '2': 0.3493877551021435, '3': 0.3493877551021435}
10:45:16 | single_host_cim_learner | INFO | ep 33 - performance: {'order_requirements': 2240000, 'container_shortage': 622649, 'operation_number': 4035090}, epsilons: {'0': 0.3477551020409223, '1': 0.3477551020409223, '2': 0.3477551020409223, '3': 0.3477551020409223}
10:45:21 | single_host_cim_learner | INFO | ep 34 - performance: {'order_requirements': 2240000, 'container_shortage': 706875, 'operation_number': 3731018}, epsilons: {'0': 0.3461224489797011, '1': 0.3461224489797011, '2': 0.3461224489797011, '3': 0.3461224489797011}
10:45:26 | single_host_cim_learner | INFO | ep 35 - performance: {'order_requirements': 2240000, 'container_shortage': 476805, 'operation_number': 4059672}, epsilons: {'0': 0.34448979591847995, '1

10:47:40 | single_host_cim_learner | INFO | ep 63 - performance: {'order_requirements': 2240000, 'container_shortage': 377861, 'operation_number': 4280589}, epsilons: {'0': 0.24156734693909476, '1': 0.24156734693909476, '2': 0.24156734693909476, '3': 0.24156734693909476}
10:47:45 | single_host_cim_learner | INFO | ep 64 - performance: {'order_requirements': 2240000, 'container_shortage': 256642, 'operation_number': 4542020}, epsilons: {'0': 0.23516734693910757, '1': 0.23516734693910757, '2': 0.23516734693910757, '3': 0.23516734693910757}
10:47:50 | single_host_cim_learner | INFO | ep 65 - performance: {'order_requirements': 2240000, 'container_shortage': 246418, 'operation_number': 4533773}, epsilons: {'0': 0.22876734693912038, '1': 0.22876734693912038, '2': 0.22876734693912038, '3': 0.22876734693912038}
10:47:55 | single_host_cim_learner | INFO | ep 66 - performance: {'order_requirements': 2240000, 'container_shortage': 251040, 'operation_number': 4303025}, epsilons: {'0': 0.222367346

10:50:15 | single_host_cim_learner | INFO | ep 94 - performance: {'order_requirements': 2240000, 'container_shortage': 288975, 'operation_number': 3916529}, epsilons: {'0': 0.04316734693949173, '1': 0.04316734693949173, '2': 0.04316734693949173, '3': 0.04316734693949173}
10:50:20 | single_host_cim_learner | INFO | ep 95 - performance: {'order_requirements': 2240000, 'container_shortage': 259727, 'operation_number': 4057942}, epsilons: {'0': 0.03676734693950453, '1': 0.03676734693950453, '2': 0.03676734693950453, '3': 0.03676734693950453}
10:50:25 | single_host_cim_learner | INFO | ep 96 - performance: {'order_requirements': 2240000, 'container_shortage': 284335, 'operation_number': 3962831}, epsilons: {'0': 0.03036734693951733, '1': 0.03036734693951733, '2': 0.03036734693951733, '3': 0.03036734693951733}
10:50:30 | single_host_cim_learner | INFO | ep 97 - performance: {'order_requirements': 2240000, 'container_shortage': 55845, 'operation_number': 4383305}, epsilons: {'0': 0.0239673469